<a href="https://colab.research.google.com/github/PorkPy/LSTM-Force-Predictor/blob/master/colab_lstm_v1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:

## Imports
import torch
import sys
import os
import numpy as np
import pandas as pd
from tqdm import tqdm
import seaborn as sns
from pylab import rcParams
import matplotlib.pyplot as plt
from matplotlib import rc
from sklearn.preprocessing import MinMaxScaler
from pandas.plotting import register_matplotlib_converters
from torch import nn, optim
import random

## Set styling parameters for figures
%matplotlib inline
%config InlineBackend.figure_format='retina'
sns.set(style='whitegrid', palette='muted', font_scale=1.2)
HAPPY_COLORS_PALETTE = ["#01BEFE", "#FFDD00", "#FF7D00", "#FF006D", "#93D30C", "#8F00FF"]
sns.set_palette(sns.color_palette(HAPPY_COLORS_PALETTE))
rcParams['figure.figsize'] = 14, 10
register_matplotlib_converters()

## Set random seed for numpy and Torch
RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)
torch.manual_seed(RANDOM_SEED)

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


Check CUDA is available

In [2]:
torch.cuda.is_available()

True

In [0]:
class CoronaVirusPredictor(nn.Module):

    def __init__(self, n_features, n_hidden, seq_len, n_layers=2, ignore_zero=True):
        super(CoronaVirusPredictor, self).__init__()

        if torch.cuda.is_available():
            device = torch.device("cuda:0")
            print("Running on the GPU")
        else:
            device = torch.device("cpu")
            print("Running on CPU")

        self.n_hidden = n_hidden
        self.seq_len = seq_len
        self.n_layers = n_layers

        self.lstm = nn.LSTM(
          input_size=n_features,
          hidden_size=n_hidden,
          num_layers=n_layers,
          dropout=0.5)


        self.linear1 = nn.Linear(in_features=n_hidden, out_features=5000)#256
        self.linear2 = nn.Linear(in_features=5000, out_features=1)
        self.linear3 = nn.Linear(in_features=512, out_features=1024)
        self.linear4 = nn.Linear(in_features=1024, out_features=1)

        #self.linear5 = nn.Linear(in_features=10000, out_features=10000)
        #self.linear6 = nn.Linear(in_features=10000, out_features=1)

        


    
    
    def reset_hidden_state(self):
        self.hidden = (
            torch.zeros(self.n_layers, self.seq_len, self.n_hidden).to(device),
            torch.zeros(self.n_layers, self.seq_len, self.n_hidden).to(device)
        )

    def forward(self, sequences):
        lstm_out, self.hidden = self.lstm(sequences.view(len(sequences), self.seq_len, -1),self.hidden)
        last_time_step = lstm_out.view(self.seq_len, len(sequences), self.n_hidden)[-1]
        y_pred = self.linear1(last_time_step)
        y_pred = self.linear2(y_pred)
        #y_pred = self.linear3(y_pred)
        #y_pred = self.linear4(y_pred)
        #y_pred = self.linear5(y_pred)
        #y_pred = self.linear6(y_pred)



        return y_pred

In [4]:
if torch.cuda.is_available():
    device = torch.device("cuda:0")
    print("Running on the GPU")
else:
    device = torch.device("cpu")
    print("Running on CPU")

Running on the GPU


In [0]:
def train_model(model):
   
    loss_fn = torch.nn.MSELoss(reduction='mean')

    optimizer = torch.optim.Adam(model.parameters(), lr=0.0005)#0.0007
        
    num_epochs =600 #600

    train_hist = np.zeros(num_epochs)
    test_hist = np.zeros(num_epochs)
    
              
    model = model.to(device)
    
    #for i in range(1): #1
        
#         for j in range(len(batches)): 
#             print("\n Epoch:", i, "Batch num:", j, '\n')
#             train_data, train_labels, test_data, test_labels = get_batches(j)       


    for t in range(num_epochs):


        for j in range(len(batches)):
            
            
            train_data, train_labels, test_data, test_labels = get_batches(j)


            if t % 100 == 0:
                model.reset_hidden_state() # Requiered because he hiddent state was forgetting too early. 

            y_pred = model(train_data)

            loss = loss_fn(y_pred.float(), train_labels)


            if test_data is not None:
                with torch.no_grad():
                    y_test_pred = model(train_data)
                    test_loss = loss_fn(y_test_pred.float(), train_labels)
                test_hist[t] = test_loss.item()

                if t % 10 == 0:  
                    print(f'Epoch {t} train loss: {loss.item()} test loss: {test_loss.item()}')
            elif t % 10 == 0:
                  print(f'Epoch {t} train loss: {loss.item()}')

            train_hist[t] = loss.item()

            optimizer.zero_grad()

            loss.backward()
            optimizer.step()




    return model.eval(), train_hist, test_hist, optimizer, t, loss_fn

In [0]:
url = 'https://raw.githubusercontent.com/PorkPy/LSTM-Force-Predictor/master/80k_data/mean_force_data.csv'

data = pd.read_csv(url)
#data = data[:80000]
main_seq = data


In [7]:
n=1000
batchesx = [data[i:i + n] for i in range(0, len(data), n)]
print(len(batchesx))

random.seed(42)
random.shuffle(batchesx)

########################
high_force_list = []
sub_list = [4,6,7,13,17,20]#,21,29,40,49,57,62,66,70,72,73,74]
for i in sub_list:
    high_force_list.append(batchesx[i])



batches = high_force_list#[:20]
val_batches = high_force_list#[20:40]
test_batches = high_force_list#[:20]
print(len(batches), len(val_batches), len(test_batches))

79
6 6 6


In [0]:
#print(len(high_force_list))
#print(high_force_list[8])

In [0]:
#print(test_batches[1])

In [0]:
def get_test_batch(batch_number):
    
    seq_size = 500

    X_test = []
    
    data = test_batches[batch_number].reset_index(drop=True)
    data= data[:seq_size]

    features = data[['joint_0', 'joint_2', 'joint_4', 'joint_5']]
    features = np.asarray(features)

    targets = data.iloc[:,-1]
    targets = np.asarray(targets)
    targets = targets.reshape(-1,1)
    
    for i in range(len(features)):           
   
        X =(features[:i+1])
        an_array = np.array(X)
        shape = np.shape(X)
        temp = np.zeros((seq_size, 4))
        temp[(seq_size-shape[0]):,:shape[1]] = an_array
        X_test.append(temp)
        y_test = targets

    
    X_test = torch.cuda.FloatTensor(X_test)
    y_test = torch.cuda.FloatTensor(y_test)
    # print(data, '\n')
    # print( X_test.shape, y_test.shape)
    jim = targets
    return(X_test, y_test, jim)
  

In [0]:
#print(get_test_batch(0))

In [0]:
def get_batches(batch_num):  
    
    seq_size = 500

    data = batches[batch_num].reset_index(drop=True)
    data= data[:seq_size]
    ################################################

    X_train = []
    X_test = []

    
    features = data[['joint_0', 'joint_2', 'joint_4', 'joint_5']]
    features = np.asarray(features)

    targets = data.iloc[:,-1]
    targets = np.asarray(targets)
    targets = targets.reshape(-1,1)


    for i in range(len(features)):           
        
        #np.random.seed(42)
       
        X =(features[:i+1])
        an_array = np.array(X)
        shape = np.shape(X)
        temp = np.zeros((seq_size, 4))
        temp[(seq_size-shape[0]):,:shape[1]] = an_array
        X_train.append(temp)
        y_train = targets
    
    data = val_batches[batch_num].reset_index(drop=True)
    data= data[:seq_size]

    features = data[['joint_0', 'joint_2', 'joint_4', 'joint_5']]
    features = np.asarray(features)

    targets = data.iloc[:,-1]
    targets = np.asarray(targets)
    targets = targets.reshape(-1,1)
    
    for i in range(len(features)):           
   
        X =(features[:i+1])
        an_array = np.array(X)
        shape = np.shape(X)
        temp = np.zeros((seq_size, 4))
        temp[(seq_size-shape[0]):,:shape[1]] = an_array
        X_test.append(temp)
        y_test = targets

    

    X_train = torch.cuda.FloatTensor(X_train) # Change data to tensors
    y_train = torch.cuda.FloatTensor(y_train)
    X_test = torch.cuda.FloatTensor(X_test)
    y_test = torch.cuda.FloatTensor(y_test)
    
    #del targets, features, data, temp, an_array
    
    #print(data)
    #print(X_train.shape, y_train.shape, X_test.shape, y_test.shape)
    return(X_train, y_train, X_test, y_test)

In [0]:
#print(get_batches(0))

In [0]:
%%time

seq_length = 500 # when using zero padding, this seq_length is a bit redundent but still has to match the zero's size.

model = CoronaVirusPredictor(
      n_features=4, 
      n_hidden=64, 
      seq_len=seq_length, 
      n_layers=2
    )

model, train_hist, test_hist, optimizer, epochs, loss = train_model(model)

Running on the GPU
Epoch 0 train loss: 119.65691375732422 test loss: 119.6642837524414
Epoch 0 train loss: 217.49432373046875 test loss: 217.4732208251953
Epoch 0 train loss: 294.4775085449219 test loss: 294.47076416015625
Epoch 0 train loss: 193.48678588867188 test loss: 193.43479919433594
Epoch 0 train loss: 311.1365051269531 test loss: 311.113525390625
Epoch 0 train loss: 489.18365478515625 test loss: 489.12701416015625
Epoch 10 train loss: 45.79402542114258 test loss: 45.09754943847656
Epoch 10 train loss: 81.16183471679688 test loss: 80.32794189453125
Epoch 10 train loss: 131.15452575683594 test loss: 130.1009521484375
Epoch 10 train loss: 12.431268692016602 test loss: 11.885086059570312
Epoch 10 train loss: 40.052032470703125 test loss: 39.95691680908203
Epoch 10 train loss: 228.9415740966797 test loss: 229.8750762939453
Epoch 20 train loss: 44.75189208984375 test loss: 44.39842224121094
Epoch 20 train loss: 81.89891815185547 test loss: 82.40676879882812
Epoch 20 train loss: 127.

In [0]:
plt.plot(train_hist, label="Training loss")
plt.plot(test_hist, label="Test loss")
#plt.ylim((0, 5))
plt.legend();

RL Controller Predictor

In [0]:
# print(Xtest[0].shape)
# print(Xtest[0])
# seq = Xtest[22].reshape(-1,500,4) #For a given move,
# pred = model(seq).to(device) # send to model and get force prediction.
# print(pred)

In [0]:

Xtest, ytest, jim = get_test_batch(0)
model.eval()
with torch.no_grad():
    

    test_seq = Xtest[0].reshape(-1,500,4)#.reshape(1,200,4) # input first sequence from trajectory/batch
    preds = [] # create a list to store predictions.
    for i in range(len(Xtest)-1): # for each sequence i in the trajectory,
        y_test_pred = model(test_seq).to(device) # send sequence to model,
        pred = torch.flatten(y_test_pred).item() # reshape the model output,
        
        preds.append(pred) # and append to the list of predictions - preds.
        new_seq = Xtest[i+1].reshape(-1,500,4)#.reshape(1,200,4) # Change sequence to the next one in the list.
        #print(test_seq, pred, '\n')
        test_seq = torch.cuda.FloatTensor(new_seq).view(1, seq_length, -1) # change sequence to a torch Tensor
        

In [0]:
#print(Xtest[0])

In [0]:
preds = np.asarray(preds)
preds = np.reshape(preds, (-1,1))

ytest2 = ytest.cpu()
ytest2 = np.asarray(ytest2)
ytest2 = np.reshape(ytest2, (-1,1))

predicted_cases = preds
true_cases = ytest2


preds = np.asarray(preds)
preds = np.reshape(preds, (-1,1))

ytest2 = ytest.cpu()
ytest2 = np.asarray(ytest2)
ytest2 = np.reshape(ytest2, (-1,1))

predicted_cases = preds
true_cases = ytest2


print(main_seq.shape)
seq = pd.DataFrame(jim)
seq = seq.reset_index(drop=True)
main_seq_reset = main_seq.reset_index(drop=True)

plt.plot(seq,label='Sequence')

print(len(predicted_cases))
plt.plot( true_cases[:,-1], label='Real Force')

plt.plot(predicted_cases[:,-1], label='Predicted Force')



#bob = (test_batches[79].reset_index(drop=True))
#print(bob.shape)
#plt.plot(bob.iloc[:500,-1])
plt.legend();




In [0]:
plt.plot(batches[0])

In [0]:
scaler = MinMaxScaler(feature_range=(-1, 1))

scaler = scaler.fit(data)

all_data = scaler.transform(data)

all_data.shape

In [0]:
tX_all, y_all = train_create_sequences( seq_length)

X_all = torch.cuda.FloatTensor(X_all)
y_all = torch.cuda.FloatTensor(y_all)

model = CoronaVirusPredictor(
  n_features=7, 
  n_hidden=512, 
  seq_len=seq_length, 
  n_layers=2
)
model, train_hist, _ = train_model(model, X_all, y_all)

In [0]:
DAYS_TO_PREDICT = 5

with torch.no_grad():
    test_seq = X_all[:1]
    preds = []
    for _ in range(DAYS_TO_PREDICT):
    
        y_test_pred = model(test_seq).to(device)
        pred = torch.flatten(y_test_pred).item()
        preds.append(pred)
        print(pred)
        new_seq = test_seq.cpu().numpy().flatten()
        new_seq = np.append(new_seq, [pred])
        new_seq = new_seq[1:]
        test_seq = torch.cuda.FloatTensor(new_seq).view(1, seq_length, -1)
        

In [0]:
predicted_cases = scaler.inverse_transform(preds).flatten()
print(predicted_cases.shape)
                                           
# print(np.shape(preds))
# preds = np.asarray(preds)
# preds = preds.reshape([5,7])
# print(np.shape(preds))

# predicted_cases = scaler.inverse_transform(preds)                                           

In [0]:
plt.plot(daily_cases, label='Historical Daily Cases')
plt.plot(predicted_cases, label='Predicted Daily Cases')
plt.legend();

